# Raster Example

## Overview

The following raster datasets uses data from the **2022 - MCAS-S data pack (1 km)** pack. It contains Australian spatial layers that cover a broad scope of interests, including:
* Biophysical (vegetation, soil, terrain, water, climate,)
* Social (household, community and population characteristics)
* Economic (land use, agricultural commodity, income, land value, rate of return).

The dataset has been prepared by the Australian Bureau of Agricultural and Resource Economics and Sciences (ABARES), Australian Bureau of Statistics (ABS), Australian Electoral Commission (AEC), Australian National University (ANU), Bureau of Meteorology (BOM), Centre for Invasive Animal Solutions, Commonwealth Scientific and Industrial Research Organisation (CSIRO), Department of Agriculture Water and the Environment (now Department of Agriculture, Fisheries and Forestry and Department of Climate Change, Energy, the Environment and Water), Environmental Systems Research Hub, Geoscience Australia, and New South Wales government.

Get started by downloading the dataset from the following [link](https://www.agriculture.gov.au/abares/aclump/multi-criteria-analysis/australian-national-map-layers).

In [0]:
%pip install geopandas==1.1.1 folium==0.20.0 rasterio==1.4.3
%restart_python

## Install Dependencies

Install Python libraries in a new notebook `.ipynb`.
* In this method, we are using a [notebook-scoped Python library](https://learn.microsoft.com/en-us/azure/databricks/libraries/notebooks-python-libraries) which does not persist between sessions.
* A cluster library is recommended if you're sharing a set of classic compute across the team.

## Upload data

Manually upload the following datasets to Databricks workspace path and modify `data_path` parameter to get started:
* `MCAS-S-Data-Pack2022/Data/Overlay/Cities` -> `cities` folder

## Load data

In [0]:
# Fetch parameters
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
data_path = dbutils.widgets.get("data_path")

In [0]:
import pandas as pd
import geopandas


def write_geodf_to_delta(gdf: geopandas.GeoDataFrame, table_name: str):
    """
    Loads data from a GeoDataFrame to a Delta table
    """
    # Convert GeoDataFrame to Pandas DataFrame
    df = pd.DataFrame(gdf)

    # Create Spark DataFrame
    spark_df = spark.createDataFrame(df)

    # Write to Delta table
    (
      spark_df
      .write.mode("overwrite")
      .option("overwriteSchema", "true")
      .saveAsTable(f"{catalog}.{schema}.{table_name}")
    )


In [0]:
# Load cities data
import geopandas as gpd

gdf_cities = gpd.read_file(f"{data_path}/cities/cities.shp")
# Convert geometry to WKT
gdf_cities['geometry'] = gdf_cities['geometry'].to_wkt()
write_geodf_to_delta(
  gdf=gdf_cities,
  table_name="cities"
)

In [0]:
import rasterio
import geopandas as gpd
from shapely.geometry import box

with rasterio.open(f"{data_path}/roads_major/dist_roads_major.tif") as src:
    bounds = src.bounds
    geom = box(*bounds)
    gdf_roads_major = gpd.GeoDataFrame({'geometry': [geom]}, crs=src.crs)
    
    gdf_roads_major['geometry'] = gdf_roads_major['geometry'].to_wkt()

    write_geodf_to_delta(
      gdf=gdf_roads_major,
      table_name="roads_major"
    )

In [0]:
import folium
from shapely import wkt
import pandas as pd


TILES = 'Cartodb Positron'

# Create a base map
_map = folium.Map(
  location=[-26.853387500000004, 133.27515449999999],
  tiles=TILES,
  zoom_start=4
)

# Add geometries to the map with tooltips
query = f"SELECT UCL_NAME11, geometry FROM {catalog}.{schema}.cities LIMIT 10000"
df_geom = spark.sql(query).toPandas()
for _, row in df_geom.iterrows():
    (city, city_geom) = (row['UCL_NAME11'], row['geometry'])
    folium.GeoJson(
      data=wkt.loads(city_geom),
      tooltip=city
    ).add_to(_map)

query = f"SELECT geometry FROM {catalog}.{schema}.roads_major LIMIT 10000"
df_geom = spark.sql(query).toPandas()
for _, row in df_geom.iterrows():
    folium.GeoJson(
      data=wkt.loads(row['geometry']),
      # tooltip=city
    ).add_to(_map)


In [0]:
# Display map
display(_map)